In [1]:
import mysql.connector
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt

In [2]:
# Connect to the MySQL Server
try:
    connection = mysql.connector.connect(
        host='localhost',        # Replace with your MySQL host (default is 'localhost')
        user='root',             # Your MySQL username
        password='anmol123',  # Your MySQL root password
        database='sugar_prices_db'  # The database you created
    )
    print("Connection to MySQL database successful!")

except mysql.connector.Error as e:
    print(f"Error: {e}")
    exit()

Connection to MySQL database successful!


## Working with the Sugar Production (Sugar Stats) Dataset

In [3]:
df_production = pd.read_excel("./monthly sugar production.xlsx")

In [4]:
# Rename columns for clarity and consistency
df_production.columns = [
    'Month', 
    'Year', 
    'Monthly Production (lakh tonnes)', 
    'Sugar Export (metric tonnes)', 
    'Sugar Import (metric tonnes)'
]

In [5]:
# Combine 'Month' and 'Year' into a single 'Date' column in production_data
df_production['Date'] = pd.to_datetime('01-' + df_production['Month'] + '-' + df_production['Year'].astype(str))
df_production['Date'] = df_production['Date'].dt.strftime(r'%Y-%m-%d')

In [6]:
# Drop the original 'Month' and 'Year' columns as they are now combined
df_production.drop(columns=['Month', 'Year'], inplace=True)

In [7]:
df_production = df_production[['Date', 'Monthly Production (lakh tonnes)', 'Sugar Export (metric tonnes)', 'Sugar Import (metric tonnes)']]

In [8]:
df_production.fillna('0',inplace=True)
df_production

C:\Users\anmol\AppData\Local\Temp\ipykernel_29160\3699836665.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_production.fillna('0',inplace=True)


,Date,Monthly Production (lakh tonnes),Sugar Export (metric tonnes),Sugar Import (metric tonnes)
0,2023-10-01,4.36,462256.0,76719.0
1,2023-11-01,43.82,669226.0,133.0
2,2023-12-01,73.02,1586842.0,77946.0
3,2023-01-01,73.79,1394166.0,73412.0
4,2023-02-01,63.49,1166624.0,48.0
...,...,...,...,...
163,2010-05-01,4.23,0,0
164,2010-06-01,0.69,0,0
165,2010-07-01,0.55,0,0
166,2010-08-01,1.13,0,0


In [10]:
# Upload data into MySQL Table
try:
    cursor = connection.cursor()
    
    insert_query = """
    INSERT INTO sugar_stats (id, monthly_date, sugar_production, sugar_export, sugar_import)
    VALUES (%s, %s, %s, %s, %s)
    """
    data_to_insert = []
    for row in df_production.itertuples(index=True):
        data_to_insert.append(tuple(row))
    
    # Step 5: Execute the Insert Query for each row
    cursor.executemany(insert_query, data_to_insert)
    connection.commit()  # Commit the transaction

    print(cursor.rowcount, "records inserted successfully.")

except Exception as e:
    print(f"Error: {e}")

Error: 1062 (23000): Duplicate entry '1' for key 'sugar_stats.PRIMARY'


In [9]:
# Confirming whether data was correctly uploaded to the SQL Server Table and pulling it for the creation of the master dataset
try:
    query = "SELECT * FROM sugar_stats;"
    df_production_sql = pd.read_sql(query, connection)
    print("Data fetched successfully!")

    # Display the DataFrame
    print("Data from MySQL Table:")
    print(df_production_sql)

except Exception as e:
    print(f"Error fetching data: {e}")

Data fetched successfully!
Data from MySQL Table:
      id monthly_date  sugar_production  sugar_export  sugar_import
0      1   2023-11-01             43.82      669226.0         133.0
1      2   2023-12-01             73.02     1586840.0       77946.0
2      3   2023-01-01             73.79     1394170.0       73412.0
3      4   2023-02-01             63.49     1166620.0          48.0
4      5   2023-03-01             42.29      921788.0       76658.0
..   ...          ...               ...           ...           ...
163  164   2010-06-01              0.69           0.0           0.0
164  165   2010-07-01              0.55           0.0           0.0
165  166   2010-08-01              1.13           0.0           0.0
166  167   2010-09-01              1.45           0.0           0.0
167  169   2023-10-01              4.36      462256.0       76719.0

[168 rows x 5 columns]


C:\Users\anmol\AppData\Local\Temp\ipykernel_29160\2248140449.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_production_sql = pd.read_sql(query, connection)


In [10]:
df_production_sql.drop(['id'], axis=1, inplace=True)
df_production_sql

,monthly_date,sugar_production,sugar_export,sugar_import
0,2023-11-01,43.82,669226.0,133.0
1,2023-12-01,73.02,1586840.0,77946.0
2,2023-01-01,73.79,1394170.0,73412.0
3,2023-02-01,63.49,1166620.0,48.0
4,2023-03-01,42.29,921788.0,76658.0
...,...,...,...,...
163,2010-06-01,0.69,0.0,0.0
164,2010-07-01,0.55,0.0,0.0
165,2010-08-01,1.13,0.0,0.0
166,2010-09-01,1.45,0.0,0.0


## Working with the Sugar Prices Dataset

In [11]:
df_prices = pd.read_excel("./Historical sugar prices_indexmundi.xlsx")

In [12]:
# Rename columns for clarity and consistency
df_prices.columns = ['Date', 'Sugar Price', 'Price Change']

In [13]:
df_prices.drop(['Price Change'], axis=1, inplace=True)

In [14]:
df_prices['Date'] = df_prices['Date'].dt.strftime(r'%Y-%m-%d')

In [15]:
df_prices.fillna('0',inplace=True)
df_prices

,Date,Sugar Price
0,2018-01-01,39.200000
1,2018-02-01,39.100000
2,2018-03-01,38.944444
3,2018-04-01,37.703704
4,2018-05-01,35.358423
...,...,...
78,2024-07-01,42.695473
79,2024-08-01,43.266667
80,2024-09-01,42.675926
81,2024-10-01,42.884444


In [16]:
# Upload data into MySQL Table
try:
    cursor = connection.cursor()
    
    insert_query = """
    INSERT INTO historical_sugar_prices (id, monthly_date, price)
    VALUES (%s, %s, %s)
    """
    data_to_insert = []
    for row in df_prices.itertuples(index=True):
        data_to_insert.append(tuple(row))
    
    # Step 5: Execute the Insert Query for each row
    cursor.executemany(insert_query, data_to_insert)
    connection.commit()  # Commit the transaction

    print(cursor.rowcount, "records inserted successfully.")

except Exception as e:
    print(f"Error: {e}")

83 records inserted successfully.


In [17]:
# Confirming whether data was correctly uploaded to the SQL Server Table and pulling it for the creation of the master dataset
try:
    query = "SELECT * FROM historical_sugar_prices;"
    df_prices_sql = pd.read_sql(query, connection)
    print("Data fetched successfully!")

    # Display the DataFrame
    print("Data from MySQL Table:")
    print(df_prices_sql)

except Exception as e:
    print(f"Error fetching data: {e}")

Data fetched successfully!
Data from MySQL Table:
    id monthly_date    price
0    1   2018-02-01  39.1000
1    2   2018-03-01  38.9444
2    3   2018-04-01  37.7037
3    4   2018-05-01  35.3584
4    5   2018-06-01  37.0690
..  ..          ...      ...
78  79   2024-08-01  43.2667
79  80   2024-09-01  42.6759
80  81   2024-10-01  42.8844
81  82   2024-11-01  41.7778
82  84   2018-01-01  39.2000

[83 rows x 3 columns]


C:\Users\anmol\AppData\Local\Temp\ipykernel_29160\3293566509.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_prices_sql = pd.read_sql(query, connection)


In [18]:
df_prices_sql.drop(['id'], axis=1, inplace=True)
df_prices_sql

,monthly_date,price
0,2018-02-01,39.1000
1,2018-03-01,38.9444
2,2018-04-01,37.7037
3,2018-05-01,35.3584
4,2018-06-01,37.0690
...,...,...
78,2024-08-01,43.2667
79,2024-09-01,42.6759
80,2024-10-01,42.8844
81,2024-11-01,41.7778


## Working with the RBI Monthly CPI Dataset

In [19]:
df_cpi = pd.read_excel("./RBI-CPI-MonthlyStats.xlsx")

In [20]:
df_cpi.columns = ['Month', 'Commodity', 'Provisional/Final', 'Rural Index', 'Rural Inflation %', 'Urban Index', 'Urban Inflation %', 'Combined Index', 'Combined Inflation %']

In [21]:
# Filtering the data to only look at the Final(non-provisional) Sugar Commodity CPI numbers
df_cpi['Commodity'] = df_cpi['Commodity'].astype(str)
df_cpi['Commodity'] = df_cpi['Commodity'].apply(str.lower)
df_cpi = df_cpi[df_cpi["Commodity"].str.contains("sugar")]
df_cpi
df_cpi = df_cpi[df_cpi["Provisional/Final"].str.lower() == 'final']
df_cpi

,Month,Commodity,Provisional/Final,Rural Index,Rural Inflation %,Urban Index,Urban Inflation %,Combined Index,Combined Inflation %
39,SEP-2024,a.1.9) sugar and confectionery,Final,131,3.48,132.9,3.5,131.6,3.46
67,AUG-2024,a.1.9) sugar and confectionery,Final,130.7,4.73,132.8,4.57,131.4,4.7
95,JUL-2024,a.1.9) sugar and confectionery,Final,130.4,5.33,132.3,5.17,131,5.22
123,JUN-2024,a.1.9) sugar and confectionery,Final,130,5.86,132.1,5.76,130.7,5.83
151,MAY-2024,a.1.9) sugar and confectionery,Final,128.9,5.74,131.3,5.72,129.7,5.7
...,...,...,...,...,...,...,...,...,...
3847,MAY-2013,a.1.9) sugar and confectionery,Final,103.7,NaN,101.9,NaN,103.1,NaN
3875,APR-2013,a.1.9) sugar and confectionery,Final,104.2,NaN,102,NaN,103.5,NaN
3903,MAR-2013,a.1.9) sugar and confectionery,Final,105,NaN,102.6,NaN,104.2,NaN
3931,FEB-2013,a.1.9) sugar and confectionery,Final,105.7,NaN,103.8,NaN,105.1,NaN


In [22]:
df_cpi['Month'] = pd.to_datetime(df_cpi['Month'])
df_cpi['Month'] = df_cpi['Month'].dt.strftime(r'%Y-%m-%d')

C:\Users\anmol\AppData\Local\Temp\ipykernel_29160\2423996511.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_cpi['Month'] = pd.to_datetime(df_cpi['Month'])


In [23]:
df_cpi = df_cpi.drop(columns=['Commodity', 'Provisional/Final'])

In [24]:
df_cpi.fillna('0',inplace=True)
df_cpi

C:\Users\anmol\AppData\Local\Temp\ipykernel_29160\1156787247.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_cpi.fillna('0',inplace=True)


,Month,Rural Index,Rural Inflation %,Urban Index,Urban Inflation %,Combined Index,Combined Inflation %
39,2024-09-01,131.0,3.48,132.9,3.5,131.6,3.46
67,2024-08-01,130.7,4.73,132.8,4.57,131.4,4.7
95,2024-07-01,130.4,5.33,132.3,5.17,131.0,5.22
123,2024-06-01,130.0,5.86,132.1,5.76,130.7,5.83
151,2024-05-01,128.9,5.74,131.3,5.72,129.7,5.7
...,...,...,...,...,...,...,...
3847,2013-05-01,103.7,0,101.9,0,103.1,0
3875,2013-04-01,104.2,0,102.0,0,103.5,0
3903,2013-03-01,105.0,0,102.6,0,104.2,0
3931,2013-02-01,105.7,0,103.8,0,105.1,0


In [26]:
# Upload data into MySQL Table
try:
    cursor = connection.cursor()
    
    insert_query = """
    INSERT INTO monthly_rbi_cpi (id, monthly_date, rural_index, rural_inflation, urban_index, urban_inflation, combined_index, combined_inflation)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    """
    data_to_insert = []
    for row in df_cpi.itertuples(index=True):
        data_to_insert.append(tuple(row))
    
    # Step 5: Execute the Insert Query for each row
    cursor.executemany(insert_query, data_to_insert)
    connection.commit()  # Commit the transaction

    print(cursor.rowcount, "records inserted successfully.")

except Exception as e:
    print(f"Error: {e}")

141 records inserted successfully.


In [29]:
# Confirming whether data was correctly uploaded to the SQL Server Table and pulling it for the creation of the master dataset
try:
    query = "SELECT * FROM monthly_rbi_cpi;"
    df_cpi_sql = pd.read_sql(query, connection)
    print("Data fetched successfully!")

    # Display the DataFrame
    print("Data from MySQL Table:")
    print(df_cpi_sql)

except Exception as e:
    print(f"Error fetching data: {e}")

Data fetched successfully!
Data from MySQL Table:
       id monthly_date  rural_index  rural_inflation  urban_index  \
0      39   2024-09-01        131.0             3.48        133.0   
1      67   2024-08-01        131.0             4.73        133.0   
2      95   2024-07-01        130.0             5.33        132.0   
3     123   2024-06-01        130.0             5.86        132.0   
4     151   2024-05-01        129.0             5.74        131.0   
..    ...          ...          ...              ...          ...   
136  3847   2013-05-01        104.0             0.00        102.0   
137  3875   2013-04-01        104.0             0.00        102.0   
138  3903   2013-03-01        105.0             0.00        103.0   
139  3931   2013-02-01        106.0             0.00        104.0   
140  3959   2013-01-01        107.0             0.00        105.0   

     urban_inflation  combined_index  combined_inflation  
0               3.50           132.0                3.46  
1  

C:\Users\anmol\AppData\Local\Temp\ipykernel_29160\1476045047.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_cpi_sql = pd.read_sql(query, connection)


In [30]:
df_cpi_sql.drop(['id', 'rural_index', 'rural_inflation', 'urban_index', 'urban_inflation', 'combined_index'], axis=1, inplace=True)
df_cpi_sql

,monthly_date,combined_inflation
0,2024-09-01,3.46
1,2024-08-01,4.70
2,2024-07-01,5.22
3,2024-06-01,5.83
4,2024-05-01,5.70
...,...,...
136,2013-05-01,0.00
137,2013-04-01,0.00
138,2013-03-01,0.00
139,2013-02-01,0.00


## Creation of final dataset

In [76]:
# Merge the two datasets on the 'Date' column
data = pd.merge(df_production_sql, df_prices_sql, on='monthly_date', how='outer')

# Sort the merged data by date
data.sort_values('monthly_date', inplace=True)

# Reset the index for a clean dataset
data.reset_index(drop=True, inplace=True)


In [77]:
# Filter rows where the Sugar Price column has non-missing values
data = data.dropna(subset=[
    # 'Monthly Production (lakh tonnes)', 
    # 'Sugar Export (metric tonnes)', 
    # 'Sugar Import (metric tonnes)', 
    'price',
    # 'CPI'
])

In [78]:
data['monthly_date'] = pd.to_datetime(data['monthly_date'])
df_cpi_sql['monthly_date'] = pd.to_datetime(df_cpi_sql['monthly_date'])

In [79]:
# Merge the cpi data with the merged dataset of production and prices on the 'Date' column
data = pd.merge(data, df_cpi_sql, left_on='monthly_date', right_on='monthly_date', how ='left')

data

,monthly_date,sugar_production,sugar_export,sugar_import,price,combined_inflation
0,2018-01-01,67.67,97131.0,321130.0,39.2000,2.77
1,2018-02-01,60.54,114127.0,0.0,39.1000,-0.17
2,2018-03-01,51.64,134766.0,59110.0,38.9444,-1.61
3,2018-04-01,28.48,159955.0,0.0,37.7037,-4.05
4,2018-05-01,9.18,218721.0,135470.0,35.3584,-8.21
...,...,...,...,...,...,...
78,2024-07-01,NaN,NaN,NaN,42.6955,5.22
79,2024-08-01,NaN,NaN,NaN,43.2667,4.70
80,2024-09-01,NaN,NaN,NaN,42.6759,3.46
81,2024-10-01,NaN,NaN,NaN,42.8844,NaN


In [80]:
#  Ensure 'Date' column is in datetime format
data['monthly_date'] = pd.to_datetime(data['monthly_date'])

# Sort the merged data by date
data.sort_values('monthly_date', inplace=True)

data.reset_index(drop=True, inplace=True)

data.head(5)

,monthly_date,sugar_production,sugar_export,sugar_import,price,combined_inflation
0,2018-01-01,67.67,97131.0,321130.0,39.2000,2.77
1,2018-02-01,60.54,114127.0,0.0,39.1000,-0.17
2,2018-03-01,51.64,134766.0,59110.0,38.9444,-1.61
3,2018-04-01,28.48,159955.0,0.0,37.7037,-4.05
4,2018-05-01,9.18,218721.0,135470.0,35.3584,-8.21


In [81]:
data.to_clipboard()

In [82]:
# Filter rows where the Sugar Price column has non-missing values
complete_data = data.dropna(subset=[
    'price',
])

In [83]:
complete_data.fillna('0',inplace=True)
complete_data

C:\Users\anmol\AppData\Local\Temp\ipykernel_29160\3282638430.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  complete_data.fillna('0',inplace=True)


,monthly_date,sugar_production,sugar_export,sugar_import,price,combined_inflation
0,2018-01-01,67.67,97131.0,321130.0,39.2000,2.77
1,2018-02-01,60.54,114127.0,0.0,39.1000,-0.17
2,2018-03-01,51.64,134766.0,59110.0,38.9444,-1.61
3,2018-04-01,28.48,159955.0,0.0,37.7037,-4.05
4,2018-05-01,9.18,218721.0,135470.0,35.3584,-8.21
...,...,...,...,...,...,...
78,2024-07-01,0,0,0,42.6955,5.22
79,2024-08-01,0,0,0,43.2667,4.7
80,2024-09-01,0,0,0,42.6759,3.46
81,2024-10-01,0,0,0,42.8844,0


In [84]:
complete_data = complete_data.round(2).reset_index(drop=True)

In [85]:
complete_data = complete_data[['monthly_date', 'price', 'sugar_production', 'sugar_export', 'sugar_import', 'combined_inflation']]

complete_data

,monthly_date,price,sugar_production,sugar_export,sugar_import,combined_inflation
0,2018-01-01,39.20,67.67,97131.0,321130.0,2.77
1,2018-02-01,39.10,60.54,114127.0,0.0,-0.17
2,2018-03-01,38.94,51.64,134766.0,59110.0,-1.61
3,2018-04-01,37.70,28.48,159955.0,0.0,-4.05
4,2018-05-01,35.36,9.18,218721.0,135470.0,-8.21
...,...,...,...,...,...,...
78,2024-07-01,42.70,0,0,0,5.22
79,2024-08-01,43.27,0,0,0,4.7
80,2024-09-01,42.68,0,0,0,3.46
81,2024-10-01,42.88,0,0,0,0


In [89]:
# Upload data into MySQL Table
try:
    cursor = connection.cursor()
    
    insert_query = """
    INSERT INTO combined_forecasting_data (id, monthly_date, sugar_price, sugar_production, sugar_export, sugar_import, cpi)
    VALUES (%s, %s, %s, %s, %s, %s, %s)
    """
    data_to_insert = []
    for row in complete_data.itertuples(index=True):
        data_to_insert.append(tuple(row))
    
    # Step 5: Execute the Insert Query for each row
    cursor.executemany(insert_query, data_to_insert)
    connection.commit()  # Commit the transaction

    print(cursor.rowcount, "records inserted successfully.")

except Exception as e:
    print(f"Error: {e}")

83 records inserted successfully.


In [90]:
# Confirming whether data was correctly uploaded to the SQL Server Table and pulling it for the creation of the master dataset
try:
    query = "SELECT * FROM combined_forecasting_data;"
    df_completed_sql = pd.read_sql(query, connection)
    print("Data fetched successfully!")

    # Display the DataFrame
    print("Data from MySQL Table:")
    print(df_completed_sql)

except Exception as e:
    print(f"Error fetching data: {e}")

Data fetched successfully!
Data from MySQL Table:
    id monthly_date  sugar_price  sugar_production  sugar_export  \
0    1   2018-02-01        39.10             60.54      114127.0   
1    2   2018-03-01        38.94             51.64      134766.0   
2    3   2018-04-01        37.70             28.48      159955.0   
3    4   2018-05-01        35.36              9.18      218721.0   
4    5   2018-06-01        37.07              1.00      327976.0   
..  ..          ...          ...               ...           ...   
78  79   2024-08-01        43.27              0.00           0.0   
79  80   2024-09-01        42.68              0.00           0.0   
80  81   2024-10-01        42.88              0.00           0.0   
81  82   2024-11-01        41.78              0.00           0.0   
82  84   2018-01-01        39.20             67.67       97131.0   

    sugar_import   cpi  
0            0.0 -0.17  
1        59110.0 -1.61  
2            0.0 -4.05  
3       135470.0 -8.21  
4       

C:\Users\anmol\AppData\Local\Temp\ipykernel_29160\1710347332.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_cpi_sql = pd.read_sql(query, connection)


In [91]:
# Close the database connection
try:
    if 'connection' in locals() and connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection closed.")
except Exception as e:
    print("Error", e)

MySQL connection closed.
